# **Исследование книжного рынка в формате цифрового сервиса электронных книг**

# **Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей, обязав население разных стран соблюдать карантинный режим и другие меры безопасности, что способствовало людей к поиску новых увлечений и хобби (книги, сериалы, рукоделие и т.д.). Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Загрузим данные и откроем файл

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

**Сперва выведем схему базы данных данных для понимания структуры и взаимосвязей между таблицами в БД**


---


![Схема базы данных.png](https://github.com/Islam95-86/ICE-RICE-image/blob/main/%D0%A1%D1%85%D0%B5%D0%BC%D0%B0%20%D0%B1%D0%B0%D0%B7%D1%8B%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.png?raw=true)

**Выведем каждую таблицу по отдельности**

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
table_books = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(table_books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [4]:
table_authors = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(table_authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [5]:
table_publishers = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(table_publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [6]:
table_ratings = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(table_ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [7]:
table_reviews = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(table_reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Сформулируем вопросы и ответим на них

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
total_books_after_2015 = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date::DATE > '2000-01-01'
'''
pd.io.sql.read_sql(sql=text(total_books_after_2015), con = con)

,count
0,819


819 книг вышло после 1 января 2000 года

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
total_reviews_ratings = '''
SELECT b.book_id,
       title,
       COUNT(DISTINCT review_id) AS total_reviews,
       ROUND(AVG(rating), 2) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id = r.book_id
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
GROUP BY b.book_id
ORDER BY total_reviews DESC
'''
pd.io.sql.read_sql(sql=text(total_reviews_ratings), con = con)

,book_id,title,total_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Книгой-лидером по количеству обзоров является книга `Twilight (Twilight #1)`(**Сумерки, часть первая**) - 7 обзоров, средний рейтинг равен 3.66 баллам. Что касается мест 2 и 3, то их занимает ряд книг с одинаковым количеством обзоров и разным рейтингом, выделим пару из них: `Water for Elephants`(**Воды слонам**) - 6 обзоров, средний рейтинг равен 3.98, `The Glass Castle`(**Стеклянный замок**) - 6 обзоров, средний рейтинг - 4.21.	

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
best_publisher = '''
SELECT publisher_id,
       publisher,
       COUNT(book_id) AS total_books
FROM books
LEFT JOIN publishers USING(publisher_id)
WHERE num_pages > 50
GROUP BY publisher_id, publisher
ORDER BY total_books DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(best_publisher), con = con)

,publisher_id,publisher,total_books
0,212,Penguin Books,42


Издательство, которое выпустило наибольшее число книг за исключением брошюр - это **Penguin Books**, 42 изданные книги

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
best_author = '''
SELECT b.author_id,
       author,
       ROUND(AVG(rating), 2) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id = r.book_id
LEFT JOIN authors AS a ON b.author_id  = a.author_id
WHERE b.book_id IN (SELECT b.book_id
                    FROM books AS b
                    LEFT JOIN ratings AS r ON b.book_id = r.book_id
                    GROUP BY b.book_id
                    HAVING COUNT(rating_id) >= 50)
GROUP BY b.author_id, author
ORDER BY avg_rating DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(best_author), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.29


`J.K. Rowling/Mary GrandPré`(**Джоан Роулинг/Мэри Грандпре(американский иллюстратор и писатель)**) - автор с самой высокой средней оценкой книг

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
most_active_users = '''
SELECT AVG(count_reviews) AS avg_count_reviews
FROM (SELECT username,
       COUNT(review_id) AS count_reviews
FROM reviews
WHERE username IN (SELECT username
                   FROM ratings
                   GROUP BY username
                   HAVING COUNT(rating_id) > 48)
GROUP BY username
ORDER BY count_reviews DESC) AS most_active_users
'''
pd.io.sql.read_sql(sql=text(most_active_users), con = con)

,avg_count_reviews
0,24.0


Среднее количество обзоров от самых активных пользователей, которые поставили больше 48 оценок, равно 24 рецензиям

## Общий вывод

**В рамках данного исследования мы выяснили:**
- количество книг, вышедших после 1 января 2000 г. вышла 819
- книга с большим количеством обзоров - **Twilight (Twilight #1)**(Сумерки, часть первая)
- издательство, издавшее самое большое количество книг - **Penguin Books**, 42 книги
- автор с самой высокой средней оценкой книг - **J.K. Rowling/Mary GrandPré**, рейтинг равен 4.29

**Рекомендации:**
- Ориентироваться на книги-бестселлеры в каждом сегменте своего жанра
- Расширить функционал сервиса, пополнив его аудиокнигами
- Ориентироваться не только на одиночных пользователей, но и создавать условия для сотрудничества с корпоративными клиентами